In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
pd.set_option('display.max_columns', 50)
plt.rcParams['font.family']='NanumBarunGothic'
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['savefig.facecolor']='white'

In [ ]:
data = pd.read_excel(r'../../data/파이널프로젝트_RAW_210329_210926.xlsx')

In [ ]:
# tax_free_amout 드롭합니다.
data = data.drop(columns=['tax_free_amount'], errors='ignore')

# 강의 데이터 확인

## course_title 는 1641 종류, NaN 16건은 무시해도 된다
결론: 총 16건, 모두 `PAYMENT DELETED` 나 `TRANSACTION` 이다. 무시.

In [ ]:
# 강의 제목 기준 1641 강의
data.course_title.nunique()

In [ ]:
# NaN은 16건이다.
data.course_title.isna().sum()

In [ ]:
# NaN건 모두 조회. 모두 PAYMENT DELETED 나 TRANSACTION 이다.
data[data.course_title.isna()]

## 전처리 하기 전 카테고리/서브카테고리 별 강의/고객/데이터 수
- 참고: 2022년 5월 기준 현재 홈페이지에 나타난 카테고리
  `['프로그래밍', '데이터사이언스', '디자인', '영상/3D', '부동산/금융', '마케팅', '업무 생산성', '투자/재테크', '부업/창업', '숏북']`
- 서브카테고리는 홈페이지와 일치한다.

In [ ]:
# {'course_title':'nunique', 'customer_id':'nunique', 'id': 'nunique', 'transaction_amount': 'sum'}
with pd.option_context('display.max_rows', None):
    display(
        data.groupby('category_title',dropna=False)
        .agg({'course_title':'nunique', 'customer_id':'nunique', 'id': 'nunique', 'transaction_amount': 'sum'})
        .sort_values('id',ascending=False).T
        .rename({'course_title':'강의 수', 'customer_id':'고객 수', 'id': '데이터 수', 'transaction_amount': 'transaction 합'})
    )
    
    display(
        data.groupby(['category_title','subcategory_title'],dropna=False)
        .agg({'course_title':'nunique', 'customer_id':'nunique', 'id': 'nunique', 'transaction_amount': 'sum'})
        .reset_index(level=[0,1]).sort_values(['category_title','id'],ascending=False).set_index(['category_title', 'subcategory_title'])
        .rename(columns={'course_title':'강의 수', 'customer_id':'고객 수', 'id': '데이터 수', 'transaction_amount': 'transaction 합'})
    )

## subcategory 일관 되지 않은 1건 채워넣기

In [ ]:
# 강의 중 1건은 subcategory가 섞여있음
print(data.groupby('course_title').subcategory_title.nunique(dropna=False).value_counts())

In [ ]:
# 같은 course_title에 subcategory_title이 섞여있는 강의 모두 출력 (1개 강의)
s = data.groupby('course_title').subcategory_title.nunique(dropna=False)
for course_title in s[s != 1].keys()[:100]:
    print(course_title, data[data.course_title == course_title].subcategory_title.unique())

In [ ]:
# 해당 강의 데이터 총 2건
data[data.course_title == '(B2B) K-Digital Training 핀테크 서비스 프론트엔드 개발 과정 1기']

In [ ]:
# 76521번 데이터에 subcategory_title만 추가하면
data.loc[data.course_title == '(B2B) K-Digital Training 핀테크 서비스 프론트엔드 개발 과정 1기', 'subcategory_title'] = '프론트엔드 개발'

# course_title이 subcategory_title를 결정하게 된다.
print("course_title -> subcategory_title:", (data.groupby('course_title').subcategory_title.nunique(dropna=False) == 1).all())

In [ ]:
# 잘 적용 되었는지 확인
data[data.course_title == '(B2B) K-Digital Training 핀테크 서비스 프론트엔드 개발 과정 1기']

## category_title 결측치 확인

In [ ]:
data[data.category_title.isna()].course_title.value_counts(dropna=False)

## category_title에 일관되지 않은 데이터 확인

In [ ]:
# 같은 course_title에 category_title이 다른 경우
s = data.groupby('course_title').category_title.nunique(dropna=False)
for course_title in s[s != 1].keys()[:100]:
    print(course_title, data[data.course_title == course_title].category_title.unique())

In [ ]:
# 해당 강좌 카테고리-시간 분포 확인
# 비중이 매우 적고 특정 시점에 몰려있으므로 단순 실수이거나 바로 변경한 것이라고 생각된다.

s = data.groupby('course_title').category_title.nunique(dropna=False)
for course_title in s[s != 1].keys()[:100]:
    plt.figure(figsize=(15,1))
    plt.title(course_title)
    sns.histplot(data[data.course_title == course_title], x='completed_at', hue='category_title',
                 binwidth=1, multiple='stack')
    plt.xlim(pd.to_datetime(['2021-03-27', '2021-09-27']))
    plt.show()

## `데이터사이언스` 와 `데이터 사이언스` 합치기

In [ ]:
# 데이터 사이언스는 위의 두 건 뿐이다.
for course_title in data[data.category_title == '데이터 사이언스'].course_title.unique():
    print(course_title)

In [ ]:
# TODO

## `업무 생산성`과 `비즈니스`?

In [ ]:
# `업무 생산성`와 `비즈니스`에 해당되는 강의를 확인한다.
# 내용상 비슷해 보이므로 합쳐도 될 것 같다.

for course_title in data[data.category_title == '비즈니스'].course_title.unique():
    print(course_title)
print()
for course_title in data[data.category_title == '업무 생산성'].course_title.unique():
    print(course_title)

In [ ]:
# `비즈니스`에 해당하는 추가 6건을 확인한다.
for course_title in data[data.category_title == '비즈니스'].course_title.unique()[3:]:
    plt.figure(figsize=(15,1))
    result = data[data.course_title == course_title]
    plt.title(f"{course_title} ({len(result)} 건)")
    sns.histplot(result, x='completed_at', hue='category_title',
                 binwidth=1, binrange=mpl.dates.date2num(pd.to_datetime(['2021-03-27', '2021-09-27'])), multiple='stack')
    plt.xlim(pd.to_datetime(['2021-03-27', '2021-09-27']))
    plt.show()

In [ ]:
# 업무생산성, 비즈니스 비중 차이
# ['지식콘텐츠', '비즈니스', '파이낸스', 'NaN', '데이터 사이언스', '관리', '일러스트']

plt.figure(figsize=(20,10))
target = data[data.category_title.isin(['업무 생산성', '비즈니스'])]
sns.histplot(target, x='completed_at', hue='category_title',
             binwidth=1,
             binrange=mpl.dates.date2num(pd.to_datetime(['2021-03-27', '2021-09-27'])),
             multiple='stack',
             #element = 'step',
             #fill = False
            )
plt.xlim(pd.to_datetime(['2021-03-27', '2021-09-27']))
plt.show()

## `부동산/금융`, `투자/재테크`, `파이낸스`?

In [ ]:
# '부동산/금융'과 '파이낸스'가 제목이 비슷해 보이므로 합쳐도 될 것 같다.

for course_title in data[data.category_title == '부동산/금융'].course_title.unique():
    print(course_title)
print()
for course_title in data[data.category_title == '투자/재테크'].course_title.unique():
    print(course_title)
print()
for course_title in data[data.category_title == '파이낸스'].course_title.unique():
    print(course_title)

In [ ]:
# ['부동산/금융', '투자/재테크', '파이낸스'] 데이터 비중 비교
# 부동산/금융 투자/재테크는 데이터가 많고 모든 시간에 분포해 있다. 현재 홈페이지에서도 확인 가능하다.

plt.figure(figsize=(20,10))
target = data[data.category_title.isin(['부동산/금융', '투자/재테크', '파이낸스'])]
sns.histplot(target, x='completed_at', hue='category_title',
             binwidth=1,
             binrange=mpl.dates.date2num(pd.to_datetime(['2021-03-27', '2021-09-27'])),
             multiple='stack',
             #element = 'step',
             #fill = False
            )
plt.xlim(pd.to_datetime(['2021-03-27', '2021-09-27']))
plt.show()

# 연습장